<a href="https://colab.research.google.com/github/microprediction/winningnotebooks/blob/main/Luce_Axiom_LLMs_Variations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install winning
!pip install pandas
!pip install scipy

# Luce's Choice Axiom versus the Standard Normal Race model
The methodology is as follows.

1.   Ask an LLM to assign probabilities $p_i$ to a set A of tokens
2.   Ask an LLM to assign probabilities to a subset $B \subset A$ of tokens

We then try to predict the subset probabilities in two ways:

1.   A simple renormalization (Luce Choice Axiom):  $p_i/(\sum_{j\in B} p_j)$
2.   The Standard Normal Race model: Set $X_i \sim N(a_i,1)$ where $a_i$ are calibrated to the $p_i$ using the ability transform.  

We then compare the errors.




## A contest model for choice

Luce is trivial. Let's just implement the second here using the `winning` package:;

In [3]:
from winning.std_calibration import std_state_price_implied_ability, STD_UNIT, STD_L, STD_SCALE, std_ability_implied_state_prices
def ability_implied_subrace_probabilities(race:dict, runners:[str])-> dict:
     #   Subrace probabilities
     probs = list(race.values())
     names = list(race.keys())
     abilities = std_state_price_implied_ability(probs, unit=STD_UNIT, L=STD_L, scale=STD_SCALE)
     sub_names = [nm for nm in names if nm in runners]
     sub_abil = [a for nm, a in zip(names,abilities) if nm in runners]
     sub_prob = implied_probabilities = std_ability_implied_state_prices(ability=sub_abil,unit=STD_UNIT, L=STD_L, scale=STD_SCALE)
     implied = dict( zip(sub_names,sub_prob) )
     return implied


race = {'red':0.5,'green':0.3,'blue':0.2}

runners = ['green','red']
implied = ability_implied_subrace_probabilities(race,runners )
implied

{'red': 0.6169905666139499, 'green': 0.38396120015303187}

## Experimental Setup...

In [78]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import pandas as pd
import numpy as np
import itertools
import os
from huggingface_hub import login

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Authenticate with Hugging Face Hub if token is available
hf_token = os.getenv("HF_TOKEN")
if hf_token:
    login(token=hf_token)

def fill_in_missing_word(sentence, exclude_words=None, top_k=20):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='pt')
    input_ids = inputs['input_ids']

    # Find the index of the masked token
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    # Get the model's predictions (logits)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits

    if exclude_words:
        # Get the IDs of the words to exclude
        exclude_ids = tokenizer.convert_tokens_to_ids(exclude_words)
        # Set their logits to a very low value
        logits[0, mask_token_index, exclude_ids] = -float('inf')

    # Apply softmax to get probabilities
    probs = torch.softmax(logits[0, mask_token_index], dim=-1)

    # Get the top_k predictions
    top_k_probs, top_k_indices = torch.topk(probs, top_k, dim=-1)

    # If there's only one mask token, adjust dimensions
    if top_k_indices.dim() == 2 and top_k_indices.size(0) == 1:
        top_k_indices = top_k_indices.squeeze(0)
        top_k_probs = top_k_probs.squeeze(0)

    top_k_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())
    top_k_probs = top_k_probs.tolist()

    # Store the top predictions in a dictionary
    predictions = dict(zip(top_k_tokens, top_k_probs))

    return predictions

def luce_check(sentence1, sentence2):
    # Get probabilities from both sentences
    probs1 = fill_in_missing_word(sentence1, top_k=100)
    probs2 = fill_in_missing_word(sentence2, top_k=10)

    # Filter out words in sentence2 not present in sentence1
    common_tokens = set(probs1.keys()).intersection(set(probs2.keys()))

    # Create filtered dictionaries with common tokens
    probs1_filtered = {token: probs1[token] for token in common_tokens}
    probs2_filtered = {token: probs2[token] for token in common_tokens}

    # Store the original scores (unnormalized) before renormalization
    original_scores = {token: probs1[token] for token in probs1_filtered}

    # Renormalize probs2 so they sum to 1
    total_prob2 = sum(probs2_filtered.values())
    probs2_normalized = {token: prob / total_prob2 for token, prob in probs2_filtered.items()}

    # Renormalize probs1 so they sum to 1
    total_prob1 = sum(probs1_filtered.values())
    probs1_normalized = {token: prob / total_prob1 for token, prob in probs1_filtered.items()}

    # Also add ability implied ...
    prob2_ability = ability_implied_subrace_probabilities(race=probs1, runners=common_tokens)


    # Merge both into a DataFrame for comparison
    df = pd.DataFrame({
        'Choice': list(probs1_normalized.keys()),
        'Original score': [original_scores[token] for token in probs1_normalized.keys()],
        'Luce score': list(probs1_normalized.values()),
        'Ability score': [ prob2_ability[token] for token in probs1_normalized.keys()],
        'Actual score': [probs2_normalized[token] for token in probs1_normalized.keys()]
    })

    # Add a column for the empirical / Luce ratio
    df['Actual/Luce'] = df['Actual score'] / df['Luce score']
    df['Actual/Ability'] = df['Actual score'] / df['Ability score']
    df['Ability RMSE'] =  np.sqrt(((df['Actual score'] - df['Ability score']) ** 2).mean())
    df['Luce RMSE'] =  np.sqrt(((df['Actual score'] - df['Luce score']) ** 2).mean())
    winner = 0 if df['Luce RMSE'].loc[0]<df['Ability RMSE'].loc[0] else 1
    df['Winner'] = winner

    df.sort_values('Luce score',inplace=True, ascending=False)

    return df


# Function to generate all sentence variations based on brackets while preserving [MASK]
def generate_variations(sentence_template):
    parts = sentence_template.split('[')
    variations = ['']
    for part in parts:
        if ']' in part:
            options, rest = part.split(']', 1)
            options = options.split('|')
            variations = [v + option + rest for v in variations for option in options]
        else:
            variations = [v + part for v in variations]

    # Ensure [MASK] remains intact
    variations = [v.replace('MASK', '[MASK]') for v in variations]
    return variations

# Function to remove optional clauses in angle brackets
def remove_angle_brackets(sentence_template):
    while '<' in sentence_template and '>' in sentence_template:
        start = sentence_template.index('<')
        end = sentence_template.index('>') + 1
        sentence_template = sentence_template[:start] + sentence_template[end:]
    return sentence_template

# Function to generate angle bracket variations
def generate_angle_variations(sentence_template):
    parts = sentence_template.split('<')
    variations = ['']
    for part in parts:
        if '>' in part:
            options, rest = part.split('>', 1)
            options = options.split('|')
            variations = [v + option + rest for v in variations for option in options]
        else:
            variations = [v + part for v in variations]
    return variations

# Function to run the Luce check for all aligned variations and concatenate results
def run_luce_analysis_aligned(sentence_templates:[str]):
    all_results = []
    for sentence_template in sentence_templates:
        sentence_variations = generate_variations(sentence_template)

        for sentence1 in sentence_variations:
            sentence1_no_brackets = remove_angle_brackets(sentence1)
            sentence2_variations = generate_angle_variations(sentence1)

            for sentence2 in sentence2_variations:
                print(sentence2)
                df = luce_check(sentence1_no_brackets, sentence2)
                if not df.empty:
                    df['Question Pair'] = f"{sentence1_no_brackets} | {sentence2}"
                    all_results.append(df)
                    print(df)

    # Concatenate all results into a single DataFrame
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
    else:
        final_df = pd.DataFrame()



    return final_df




/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a Bert

## Examples

In [77]:
sentence_templates = [ "My favourite <Western|Eastern|Northern|Southern> state in the [U.S.|U.K.] is [MASK] and I try to visit once a year.",
                      f"The <infectious|old age|hereditary|deficiency|physiological> disease I fear most is known as {tokenizer.mask_token} and my uncle got it.",
                       f"My favourite type of <citrus|stone|tropical> fruit is called a {tokenizer.mask_token} and I eat one of them every day.",
                       f"My favourite <alcoholic|caffienated> drink is called {tokenizer.mask_token} and I drink it when I can",
                       f"My favorite <farm|> animal is called a {tokenizer.mask_token} and I like them a lot.",
                       f"My favorite <predatory|waterfowl|wading|song|sea> bird is a {tokenizer.mask_token} and I like to watch them.",
                       f"My preferred mode of <public|private> transportation is called {tokenizer.mask_token} and it takes me from A to B.",
                       f"My favourite <U.K.|American|Japanese> car brand is called {tokenizer.mask_token} and they make a lot of cars",
                       f"The <winter|summer> sport that draws the biggest crowds is called {tokenizer.mask_token}, and it is fun to go and watch.",
                       f"My favourite <object-oriented|classic|low-level> programming language is called {tokenizer.mask_token} and I write programs with it all the time.",
                       f"I like to drink <hot|cold> {tokenizer.mask_token} in the morning or the evening.",
                       f"My preferred <small|reptilian> pet is a {tokenizer.mask_token}, they make great companions.",
                       f"My favorite <girl's|boys's> baby name is {tokenizer.mask_token}, and I always associate it with family.",
                       f"My favorite age for <middle school|elementary school|high scho kids is {tokenizer.mask_token}, and my daughter is that age"
                       ]
final_results = run_luce_analysis_aligned(sentence_templates)



My favourite <Western|Eastern|Northern|Southern> state in the [U.S.|U.K.] is [MASK] and I try to visit once a year.
My favourite Western state in the U.S. is [MASK] and I try to visit once a year.
       Choice  Original score  Luce score  Ability score  Actual score  \
1  california        0.108853    0.298702       0.253782      0.201740   
8     arizona        0.067399    0.184950       0.171971      0.100027   
4       texas        0.055019    0.150976       0.145925      0.076751   
2    colorado        0.031841    0.087376       0.093829      0.078146   
3      oregon        0.028873    0.079231       0.086702      0.085659   
6    oklahoma        0.019698    0.054052       0.063678      0.066355   
7      nevada        0.016581    0.045500       0.055442      0.101803   
9     montana        0.015908    0.043653       0.053601      0.142662   
0       idaho        0.010460    0.028704       0.038248      0.049947   
5     wyoming        0.009787    0.026856       0.036224      0

Looking at the results...

In [ ]:
print(f"Total inferred probabilities: {len(final_results)}")
print(f"The Standard Normal Race beats Luce Choice Axiom {int(final_results['Winner'].mean()*100)}% of the time")



Total inferred probabilities: 415
The Standard Normal Race beats Luce Choice Axiom 85% of the time


In [74]:
sentence_templates = [
    f"My favourite <classical|rock|jazz|pop|hip-hop> musician's last name is {tokenizer.mask_token}, and I listen to them every day.",
    f"I love to eat <Italian|Chinese|Mexican|Indian|Thai> food, and my favourite dish is called {tokenizer.mask_token}.",
    f"The <morning|afternoon|evening|night> exercise I enjoy most is called {tokenizer.mask_token}, and it helps me feel good.",
      f"My favourite <Android|iOS|Windows|macOS|Linux> app is named {tokenizer.mask_token}; I use it all the time.",
      f"I enjoy watching <comedy|drama|horror|action|animated> movies; my favorite character is named {tokenizer.mask_token} and [he|she] is great.",
    f"The <positive|negative|neutral> emotion I feel most often is {tokenizer.mask_token}; it affects my daily life.",
       f"My favourite <domestic|international> one word airline is called {tokenizer.mask_token}; they provide great service.",
        f"My favourite <string|wind|percussion|keyboard> instrument is the {tokenizer.mask_token}; I love its sound.",
    f"My favourite <indoor|outdoor> hobby is {tokenizer.mask_token}; I spend a lot of time on it.",
    f"My favourite <European|Asian|African|American> capital city is {tokenizer.mask_token}; I would love to visit.",
      f"My favourite <classic|modern> painter's name is {tokenizer.mask_token} and [he|she] is a genious.",
    f"I like to drive <compact|sedan|SUV|electric> cars. My favourite brand is called {tokenizer.mask_token} and they are [comfortable|fast|safe].",
    f"I enjoy <team|individual> sports. My favourite is called {tokenizer.mask_token} and it's [exciting|relaxing|] to play.",
         f"My preferred <wired|wireless> headphone brand is called {tokenizer.mask_token}; they have great sound quality.",
    f"The <domestic|international> news channel I watch is called {tokenizer.mask_token}; they provide reliable information.",
    f"I prefer <vegetarian|non-vegetarian|vegan|pescatarian> dishes. My favourite dish is called {tokenizer.mask_token}, and it is [delicious|healthy|unhealthy but tasty]",
   f"My favourite <fiction|non-fiction|poetry|drama> author's name is {tokenizer.mask_token}, I read all their books.",
    f"I enjoy <kids|adult> board games. My favourite is called {tokenizer.mask_token}, and it is fun.",
     f"My favorite <US|European|Asian|African> historical figure's name is {tokenizer.mask_token}; they changed the world.",
    f"My favourite type of <small|large|medium> dog breed is called {tokenizer.mask_token}; they make great pets.",
    f"I like <classical|modern|abstract|realism> art; my favorite artist's name is {tokenizer.mask_token}.",
    f"My favorite <city|country|beach|mountain> vacation city is called {tokenizer.mask_token}; I go there often.",
       f"I love <baking|cooking|grilling|frying> dishes. My favourite is called {tokenizer.mask_token}; it's my specialty.",
    f"My favourite <local|international> cuisine is called {tokenizer.mask_token}; it's incredibly flavorful.",
    f"I like to exercise <indoors|outdoors>. My favourite exercise is called {tokenizer.mask_token}; it keeps me fit.",
    f"I love <spring|summer|autumn|winter> festivals. My favourite is called {tokenizer.mask_token} and it was so much fun",
    f"My favorite <desktop|laptop|tablet|smartphone> brand is called {tokenizer.mask_token}; they make reliable devices.",
    f"My favourite <local|international> charity is called {tokenizer.mask_token}; I support their cause." ]

final_results = run_luce_analysis_aligned(sentence_templates)
print(f"The Standard Normal Race beats Luce Choice Axiom {int(final_results['Winner'].mean()*100)}% of the time")


My favourite <classical|rock|jazz|pop|hip-hop> musician's last name is [MASK], and I listen to them every day.
My favourite classical musician's last name is [MASK], and I listen to them every day.
    Choice  Original score  Luce score  Ability score  Actual score  \
0    james        0.009332    0.187251       0.155600      0.142140   
8     jack        0.008118    0.162893       0.142876      0.080745   
6    david        0.006659    0.133622       0.126445      0.169818   
7     john        0.006564    0.131698       0.125331      0.130371   
1  michael        0.004670    0.093709       0.101770      0.081510   
3    peter        0.004178    0.083832       0.094968      0.092729   
4  richard        0.003923    0.078725       0.091304      0.090586   
5   thomas        0.003208    0.064377       0.080715      0.099254   
2   george        0.003184    0.063893       0.080326      0.112848   

   Actual/Luce  Actual/Ability  Ability RMSE  Luce RMSE  Winner  \
0     0.759086        0.

In [80]:
print(f"The Standard Normal Race beats Luce Choice Axiom {int(final_results['Winner'].mean()*100)}% of the time")
print(f"Total inferred probabilities: {len(final_results)}")


The Standard Normal Race beats Luce Choice Axiom 80% of the time
Total inferred probabilities: 411
